In [2]:
import pypsa
import numpy as np
import pandas as pd
import re
import pickle
from pathlib import Path
from _helpers import calculate_annual_investment, calculate_annuity
from _helpers import extract_technology, extract_unit

from _helpers import configure_logging
import logging
logger = logging.getLogger(__name__)

In [3]:
def create_network():
    """Create the pypsa network scaffolding for the ESC."""
    
    # Modify PyPSA 'Link' component to allow for 2 output busses by overwriting component_attrs
    # c.f. https://www.pypsa.org/examples/chp-fixed-heat-power-ratio.html
    
    # Load additional components
    with open(snakemake.input["additional_components"], "rb") as f:
        override_component_attrs = pickle.load(f)

    # Create network with modified link-component
    network = pypsa.Network(override_component_attrs=override_component_attrs)

    # Load network components from csv files
    network.import_from_csv_folder(snakemake.input["network"])
    
    # Equally weighted snapshots, year defined via config
    year = snakemake.config["scenario"]["year"]
    # Handle leap year by dropping 29th of Februray
    snapshots = pd.date_range(str(year),str(year+1), freq="H", closed="left")
    snapshots = filter(lambda x: not((x.month==2) & (x.day==29)), list(snapshots))
    snapshots = [x.strftime('%Y-%m-%d %H:%M:%S') for x in snapshots]
    network.set_snapshots(snapshots=snapshots)
    
    return network

def attach_efficiencies(network):
    """Attach dedicated efficiencies from file.

    The efficiencies are from an additional csv file and added to the links in the pypsa network
    Format for efficiencies.csv file:
    * "from" and "to" must substrings of the bus names
    * "process" must be a substring of the name of the link

    Return
    ------
    network : pypsa.network
        network with external efficiencies attached to all links.

    """
    efficiencies = pd.read_csv(snakemake.input["efficiencies"])

    def get_efficiency(src_bus, tar_bus):

        src = extract_technology(src_bus)
        tar = extract_technology(tar_bus)

        efficiency = efficiencies[(efficiencies['to'] == tar) & (efficiencies['from'] == src)]

        if efficiency.empty is True:
            logger.warning(f"No efficiency found for link between {src_bus} and {tar_bus}.")
            return 0
        else:
            return efficiency['efficiency'].item()

    links = network.links

    for idx, row in links.iterrows():

        lead_efficiency = get_efficiency(row['bus0'], row['bus1'])
        links.loc[idx, 'efficiency'] = lead_efficiency

        additional_buses = {c for c in links.columns if c.startswith('bus') and row[c] != ""}-{'bus0','bus1'}
        for b in additional_buses:

            # by design decision all buses busn (n>1, e.g. bus2, bus3, ...) all contribute to
            # the output to bus1, e.g. bus2 feeds into bus1
            # Efficiencies are provided for the conversion from bus2 to bus1
            # and are thus weighted by the primary efficiency of bus1
            # Efficiencies are have to become negative to correctly account for the flow.
            follow_efficiency = (-1) * lead_efficiency / get_efficiency(row[b], row['bus1'])

            links.loc[idx, 'efficiency'+b.replace('bus','')] = follow_efficiency

    return network

In [4]:
def attach_costs(network):
    """
    Attach the overnight investment costs (capital costs) to the network.
    
    Costs are calculated from investment costs and FOM using EAC method
    and wacc as specified via config/snakemake.input files.
    Components name need to follow the scheme '<name> (<exp|imp>)'
    where '<name>' must correspond to the component in the costs.csv file.    
    """
    wacc = pd.read_csv(snakemake.input['wacc'], comment='#', index_col="region")
    wacc = wacc.loc[snakemake.wildcards["from"], snakemake.config["scenario"]["wacc"]]
    
    costs = pd.read_csv(snakemake.input['costs'], index_col=['technology','parameter'])  
        
    def attach_component_costs(network, component):    

        components = getattr(network, component)
        for idx, row in components.iterrows():

            try:
                tech = costs.loc[extract_technology(idx)]
            except KeyError:
                logger.info(f"No cost assumptions found for {idx}.")
                continue

            # Compare units between bus and cost data; scale investment on-demand
            prefix_cost_unit = tech.loc['investment']['unit'].split("/")[1][0]

            if component=='stores':
                prefix_bus_unit = network.buses.loc[row['bus']]['unit'][0]
            elif component=='links':
                prefix_bus_unit = network.buses.loc[row['bus0']]['unit'][0]

            investment_factor = -1

            if prefix_bus_unit == prefix_cost_unit:
                investment_factor = 1.
            elif prefix_bus_unit == "M" and prefix_cost_unit == "k":
                investment_factor = 1.e3
            else:
                logger.error(f"Cannot scale between {prefix_bus_unit} and {prefix_cost_unit} for {idx} costs.")

            # Some technologies are without FOM values
            # (e.g. battery capacity where FOM is attributed to the link/inverter/charger capacities)
            try:
                fom = tech.loc['FOM','value']
            except KeyError:
                logger.info(f"No FOM for {idx}, assuming 0%.")
                fom = 0.

            capital_cost = calculate_annuity(tech.loc['investment','value']*investment_factor,
                                             fom, tech.loc['lifetime','value'], wacc)
            components.loc[idx, 'capital_cost'] = capital_cost
        
        return network

    network = attach_component_costs(network, 'links')
    network = attach_component_costs(network, 'stores')
    
    return network
    

In [5]:
if __name__ == "__main__":

    configure_logging(snakemake)
    
    Path(snakemake.output["network"]).mkdir(parents=True,exist_ok=True)
    
    network = create_network()
    network = attach_efficiencies(network)
    network = attach_costs(network)
    

    # Save network as csv because netcdf is not working
    # see https://github.com/PyPSA/PyPSA/issues/204
    network.export_to_csv_folder(snakemake.output["network"])